In [1]:
# # # Error: Path resolution
# # %%bash

# # outd="$(pwd)"
# # cgud="../structures/charmm-gui/jobid-0147660438"

# # cd "${cgud}"
# # echo "$(pwd)"
# # python openmm/openmm_run.py \
# #     -i "$(pwd)/step5_production.inp" \
# #     -p "$(pwd)/step3_pbcsetup.psf" \
# #     -c "$(pwd)/step3_pbcsetup.crd" \
# #     -b "$(pwd)/step3_pbcsetup.str" \
# #     -t "$(pwd)/toppar.str" \
# #     -odcd "${outd}/3vte_apo.cgui.dcd" \
# #     -opdb "${outd}/3vte_apo.cgui.pdb" \
# #     -hmr 

In [2]:
%%bash

# # Parameterize CBGA COO(-)
# antechamber \
#     -fi pdb -i ../structures/CBGA.COO.DOCK.1.61.pdb \
#     -fo mol2 -o ./CBGA.COO.1.61.gaff.mol2 \
#     -rn LIG -c bcc -at gaff2 -j 4 -pf y \
#     -nc -1
# parmchk2 \
#     -i ./CBGA.COO.1.61.gaff.mol2 \
#     -f mol2 -o CBGA.COO.1.61.frcmod -s 2

# # Make local copies
obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -osdf > LIG.sdf
obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -opdb > LIG.pdb

1 molecule converted
1 molecule converted


In [3]:
%%capture

import mdtraj as md
import parmed as pmd

from simtk import openmm, unit
from simtk.openmm import app, LangevinIntegrator
from simtk.openmm.app import PME, NoCutoff, HBonds

from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator
from openforcefield.typing.engines.smirnoff import ForceField

# # Load the Amber restart from CHARMM-GUI's glycan modeller
prmslug = '../structures/charmm-gui/jobid-0147660438/amber/step3_input'
prm3VTE = pmd.amber.AmberParm(
    f'{prmslug}.parm7', f'{prmslug}.rst7')
apo_system = prm3VTE.createSystem()

# # Apo-enzyme in octahedral water, d=10
pmd3VTE = pmd.openmm.load_topology(
    prm3VTE.topology,
    apo_system,
    xyz=prm3VTE.positions)

# # Load Parsley force field
offLIG_ff = ForceField('openff-1.2.1.offxml')

# # Parameterize CBGA
molCBGA = Molecule('LIG.sdf')
ligand_system = offLIG_ff.create_openmm_system(
    molCBGA.to_topology())

# # Docked CBGA carboxylate
ommCBGA = md.load('LIG.pdb')
ommCBGA.xyz[:,:,1:3] += [0.65,8.65] # Hacky realignment
# ommCBGA.xyz[:,:,0:3] *= [-1,-1,-1] # Hacky realignment
pmdCBGA = pmd.openmm.load_topology(
    ommCBGA.top.to_openmm(),
    ligand_system,
    xyz=ommCBGA.openmm_positions(0))

In [4]:
# # Holo-enzyme
pmdCPLX = pmd3VTE + pmdCBGA

pmdCPLX.save('complex.pdb', overwrite=True)
pmdCPLX.save('complex.rst7', overwrite=True)

pmdCPLX.visualize()

NGLWidget()

In [5]:
# # Dummy simulation
integrator = LangevinIntegrator(
    302.5*unit.kelvin, 
    1/unit.picosecond, 
    0.002*unit.picoseconds)

# TODO Periodic box, see omm_readparams.py
# TODO CHARMM-GUI params, see omm_readparams.py
complex_system = pmdCPLX.createSystem(
    """<Extracted CHARMM-GUI parameters>""")

simulation = app.Simulation(
  pmdCPLX.topology, complex_system, integrator)
simulation.context.setPositions(pmdCPLX.positions)
simulation.minimizeEnergy()

ParameterError: Cannot find necessary parameters